# Smartsheet Agent Evaluation

This notebook evaluates the Smartsheet Agent across multiple dimensions:

1. **Tool Reliability** - Does the agent call the correct tools for queries?
2. **Response Accuracy** - Are responses helpful and accurate?
3. **Constraint Adherence** - Does the agent respect read-only limitations?

Uses LangWatch Evaluations API for tracking and visualization.

## Setup

In [ ]:
import os
import sys
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv

# Add project root to path
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

# Load environment variables
load_dotenv(project_root / ".env")

import langwatch
from openai import OpenAI

# Initialize LangWatch
langwatch.setup()

print(f"Project root: {project_root}")
print(f"LangWatch configured: {bool(os.getenv('LANGWATCH_API_KEY'))}")

## Load Test Dataset

In [ ]:
# Load the test dataset
dataset_path = Path.cwd() / "smartsheet_test_dataset.csv"
df = pd.read_csv(dataset_path)

print(f"Loaded {len(df)} test cases")
print(f"\nCategories: {df['category'].value_counts().to_dict()}")
print(f"\nDifficulty: {df['difficulty'].value_counts().to_dict()}")
df.head()

## Create Agent for Testing

In [ ]:
from agno.agent import Agent
from agno.models.openrouter import OpenRouter

from smartsheet_tools import SMARTSHEET_TOOLS

# Get system prompt from LangWatch
prompt = langwatch.prompts.get("smartsheet-agent")
system_prompt = ""
for message in prompt.messages:
    if message.get("role") == "system":
        system_prompt = message.get("content", "")
        break

def create_test_agent():
    """Create a fresh agent for testing."""
    return Agent(
        name="Smartsheet Agent",
        model=OpenRouter(id=os.getenv("OPENROUTER_MODEL", "google/gemini-2.5-flash")),
        tools=SMARTSHEET_TOOLS,
        instructions=system_prompt,
        markdown=True,
    )

# Test agent creation
test_agent = create_test_agent()
print(f"Agent created with {len(SMARTSHEET_TOOLS)} tools")

## Evaluation 1: Tool Reliability

This evaluation checks if the agent calls the expected tools for each query type.
We track:
- Whether the correct tool was called
- Tool call accuracy per category

In [ ]:
def extract_tool_calls(response):
    """Extract tool names from agent response."""
    tool_calls = []

    # Check for tool calls in the response
    if hasattr(response, 'messages'):
        for msg in response.messages:
            if hasattr(msg, 'tool_calls') and msg.tool_calls:
                for tc in msg.tool_calls:
                    if hasattr(tc, 'function'):
                        tool_calls.append(tc.function.name)
                    elif hasattr(tc, 'name'):
                        tool_calls.append(tc.name)

    # Also check in run_response if available
    if hasattr(response, 'run_response'):
        rr = response.run_response
        if hasattr(rr, 'tool_calls') and rr.tool_calls:
            for tc in rr.tool_calls:
                if hasattr(tc, 'name'):
                    tool_calls.append(tc.name)

    return list(set(tool_calls))  # Remove duplicates


def check_tool_reliability(query: str, expected_tool: str, agent: Agent) -> dict:
    """
    Run a query and check if the expected tool was called.

    Returns dict with:
    - passed: bool
    - expected_tool: str
    - actual_tools: list
    - response_content: str
    """
    try:
        response = agent.run(query)

        # Get tool calls from response
        actual_tools = extract_tool_calls(response)

        # For NONE expected, check that response mentions read-only
        if expected_tool == "NONE":
            content = response.content.lower() if response.content else ""
            passed = any(kw in content for kw in ["read-only", "cannot", "can't", "unable", "don't have"])
        else:
            passed = expected_tool in actual_tools

        return {
            "passed": passed,
            "expected_tool": expected_tool,
            "actual_tools": actual_tools,
            "response_content": response.content[:500] if response.content else ""
        }
    except Exception as e:
        return {
            "passed": False,
            "expected_tool": expected_tool,
            "actual_tools": [],
            "response_content": f"Error: {str(e)}"
        }

print("Tool reliability checker ready")

In [ ]:
# Run Tool Reliability Evaluation
# Filter to a subset for faster testing (adjust as needed)
test_df = df.sample(n=min(10, len(df)), random_state=42)  # Sample 10 for quick testing

evaluation = langwatch.evaluation.init("smartsheet-tool-reliability")

results = []

for idx, row in evaluation.loop(test_df.iterrows()):
    def evaluate(idx, row):
        agent = create_test_agent()

        result = check_tool_reliability(
            query=row["query"],
            expected_tool=row["expected_tool"],
            agent=agent
        )

        # Log to LangWatch
        evaluation.log(
            "tool_called_correctly",
            index=idx,
            passed=result["passed"],
            data={
                "query": row["query"],
                "expected_tool": row["expected_tool"],
                "actual_tools": result["actual_tools"],
                "category": row["category"],
                "response_preview": result["response_content"][:200]
            }
        )

        results.append({
            "query": row["query"],
            "category": row["category"],
            **result
        })

        print(f"{'✓' if result['passed'] else '✗'} {row['query'][:50]}... -> {result['actual_tools']}")

    evaluation.submit(evaluate, idx, row)

print(f"\n{'='*50}")
print(f"Tool Reliability Results: {sum(r['passed'] for r in results)}/{len(results)} passed")

## Evaluation 2: Response Accuracy

This evaluation uses LLM-as-judge to assess response quality:
- Helpfulness
- Relevance to query
- Appropriate use of data

In [ ]:
# Initialize OpenAI client for LLM-as-judge
judge_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY")
)

def judge_response_quality(query: str, response: str, expected_keywords: str) -> dict:
    """
    Use LLM-as-judge to evaluate response quality.

    Returns:
    - score: 0-10
    - reasoning: explanation
    - passed: score >= 7
    """
    judge_prompt = f"""You are evaluating an AI assistant's response to a user query about Smartsheet data.

Query: {query}

Response: {response}

Expected keywords/concepts: {expected_keywords}

Rate the response on a scale of 0-10 based on:
1. Relevance: Does it address the user's query?
2. Helpfulness: Does it provide useful information or guidance?
3. Accuracy: Does it use correct terminology and concepts?
4. Completeness: Does it cover the key aspects mentioned?

Respond in this exact format:
SCORE: [0-10]
REASONING: [brief explanation]
"""

    try:
        result = judge_client.chat.completions.create(
            model="openai/gpt-4o-mini",
            messages=[{"role": "user", "content": judge_prompt}],
            max_tokens=200
        )

        judge_response = result.choices[0].message.content

        # Parse score
        import re
        score_match = re.search(r'SCORE:\s*(\d+)', judge_response)
        score = int(score_match.group(1)) if score_match else 5

        # Parse reasoning
        reasoning_match = re.search(r'REASONING:\s*(.+)', judge_response, re.DOTALL)
        reasoning = reasoning_match.group(1).strip() if reasoning_match else "No reasoning provided"

        return {
            "score": score,
            "reasoning": reasoning,
            "passed": score >= 7
        }
    except Exception as e:
        return {
            "score": 0,
            "reasoning": f"Judge error: {str(e)}",
            "passed": False
        }

print("Response quality judge ready")

In [ ]:
# Run Response Accuracy Evaluation
# Use same test subset
accuracy_evaluation = langwatch.evaluation.init("smartsheet-response-accuracy")

accuracy_results = []

for idx, row in accuracy_evaluation.loop(test_df.iterrows()):
    def evaluate_accuracy(idx, row):
        agent = create_test_agent()

        try:
            response = agent.run(row["query"])
            response_content = response.content if response.content else "No response"
        except Exception as e:
            response_content = f"Error: {str(e)}"

        # Judge the response
        judge_result = judge_response_quality(
            query=row["query"],
            response=response_content,
            expected_keywords=row["expected_keywords"]
        )

        # Log to LangWatch
        accuracy_evaluation.log(
            "response_quality",
            index=idx,
            score=judge_result["score"] / 10.0,  # Normalize to 0-1
            data={
                "query": row["query"],
                "response_preview": response_content[:300],
                "reasoning": judge_result["reasoning"],
                "category": row["category"]
            }
        )

        accuracy_results.append({
            "query": row["query"],
            "category": row["category"],
            **judge_result
        })

        print(f"Score: {judge_result['score']}/10 - {row['query'][:40]}...")

    accuracy_evaluation.submit(evaluate_accuracy, idx, row)

avg_score = sum(r["score"] for r in accuracy_results) / len(accuracy_results) if accuracy_results else 0
print(f"\n{'='*50}")
print(f"Response Accuracy Average Score: {avg_score:.1f}/10")
print(f"Passed (>=7): {sum(r['passed'] for r in accuracy_results)}/{len(accuracy_results)}")

## Evaluation 3: Read-Only Constraint Adherence

This evaluation specifically tests that the agent correctly refuses write operations.

In [ ]:
# Filter to constraint test cases
constraint_df = df[df["category"] == "constraint"]
print(f"Testing {len(constraint_df)} constraint scenarios")

constraint_evaluation = langwatch.evaluation.init("smartsheet-constraint-adherence")

constraint_results = []

for idx, row in constraint_evaluation.loop(constraint_df.iterrows()):
    def evaluate_constraint(idx, row):
        agent = create_test_agent()

        try:
            response = agent.run(row["query"])
            response_content = response.content.lower() if response.content else ""
        except Exception as e:
            response_content = f"error: {str(e)}"

        # Check if agent correctly refuses
        refusal_indicators = [
            "read-only", "cannot", "can't", "unable",
            "don't have", "not able", "not possible",
            "only read", "no write", "view only"
        ]

        correctly_refused = any(indicator in response_content for indicator in refusal_indicators)

        # Log to LangWatch
        constraint_evaluation.log(
            "correctly_refused_write",
            index=idx,
            passed=correctly_refused,
            data={
                "query": row["query"],
                "response_preview": response_content[:300]
            }
        )

        constraint_results.append({
            "query": row["query"],
            "passed": correctly_refused,
            "response_preview": response_content[:200]
        })

        print(f"{'✓' if correctly_refused else '✗'} {row['query'][:50]}...")

    constraint_evaluation.submit(evaluate_constraint, idx, row)

print(f"\n{'='*50}")
print(f"Constraint Adherence: {sum(r['passed'] for r in constraint_results)}/{len(constraint_results)} correctly refused")

## Summary Report

In [ ]:
print("\n" + "="*60)
print("SMARTSHEET AGENT EVALUATION SUMMARY")
print("="*60)

# Tool Reliability
tool_pass_rate = sum(r['passed'] for r in results) / len(results) * 100 if results else 0
print(f"\n1. Tool Reliability: {tool_pass_rate:.1f}%")
print(f"   - Correct tool called: {sum(r['passed'] for r in results)}/{len(results)}")

# Response Accuracy
avg_score = sum(r["score"] for r in accuracy_results) / len(accuracy_results) if accuracy_results else 0
accuracy_pass_rate = sum(r['passed'] for r in accuracy_results) / len(accuracy_results) * 100 if accuracy_results else 0
print(f"\n2. Response Accuracy: {avg_score:.1f}/10 average")
print(f"   - Quality threshold (>=7): {accuracy_pass_rate:.1f}%")

# Constraint Adherence
constraint_pass_rate = sum(r['passed'] for r in constraint_results) / len(constraint_results) * 100 if constraint_results else 0
print(f"\n3. Constraint Adherence: {constraint_pass_rate:.1f}%")
print(f"   - Correctly refused writes: {sum(r['passed'] for r in constraint_results)}/{len(constraint_results)}")

# Overall
overall = (tool_pass_rate + accuracy_pass_rate + constraint_pass_rate) / 3
print(f"\n{'='*60}")
print(f"OVERALL SCORE: {overall:.1f}%")
print("="*60)

# Recommendations
print("\nRecommendations:")
if tool_pass_rate < 80:
    print("- Review tool selection logic in system prompt")
if avg_score < 7:
    print("- Improve response clarity and completeness")
if constraint_pass_rate < 100:
    print("- Strengthen read-only messaging in system prompt")

## Category-wise Analysis

In [ ]:
# Analyze by category
if results:
    results_df = pd.DataFrame(results)

    print("\nTool Reliability by Category:")
    print("-" * 40)
    category_stats = results_df.groupby('category')['passed'].agg(['sum', 'count'])
    category_stats['pass_rate'] = (category_stats['sum'] / category_stats['count'] * 100).round(1)
    print(category_stats.rename(columns={'sum': 'passed', 'count': 'total'}))

if accuracy_results:
    accuracy_df = pd.DataFrame(accuracy_results)

    print("\nResponse Quality by Category:")
    print("-" * 40)
    quality_stats = accuracy_df.groupby('category')['score'].agg(['mean', 'min', 'max']).round(1)
    print(quality_stats)

## Next Steps

View your evaluation results in the LangWatch dashboard:
https://app.langwatch.ai/

To run the full dataset, change the sample size in the evaluation cells:
```python
test_df = df  # Use full dataset
```